In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from 指标计算 import *

plt.rcParams['font.sans-serif']=['SimHei']  # 指定默认字体为黑体
plt.rcParams['axes.unicode_minus']=False   # 解决保存图像是负号'-'显示为方块的问题

In [ ]:
data = cal_main('../DB/GFEX.csv', 'SI')

In [22]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

# 读取数据
data = cal_main('../DB/GFEX.csv', 'SI')
rawData = data.copy()
# 定义特征和目标变量
features = ['open', 'high', 'low', 'close', 'volume', 'open_interest', 'turnover']
target = 'next_change'
# 对每一列数据进行标准化
scaler = StandardScaler()
data[features] = scaler.fit_transform(data[features])
# 计算涨跌情况
data['change'] = data['close'].diff().shift(-1)
data['next_change'] = data['change'].apply(lambda x: 1 if x > 0 else 0)
# 删除最后一个样本（目标变量为NaN）
data.drop(data.index[-1], inplace=True)
# 划分数据集为训练集和测试集
train_size = int(len(data) * 0.75)
train_data = data.iloc[:train_size]
test_data = data.iloc[train_size:]
# 创建随机森林模型
rf = RandomForestClassifier(n_estimators=1, max_depth=3, random_state=0)
# 训练模型
rf.fit(train_data[features], train_data[target])
# 进行预测
train_predictions = rf.predict(train_data[features])
test_predictions = rf.predict(test_data[features])
# 计算模型准确率
train_accuracy = accuracy_score(train_data[target], train_predictions)
test_accuracy = accuracy_score(test_data[target], test_predictions)

print(f'训练集准确率:', train_accuracy)
print(f'测试集准确率:', test_accuracy)

训练集准确率: 0.7457627118644068
测试集准确率: 0.65


In [23]:
pre = np.hstack((train_predictions, test_predictions))
signal = np.reshape(pre, (-1, 1))
signalNew = pd.DataFrame(signal)
signalNew['信号'] = '无'
signalNew = signalNew.rename(columns={0: '预测'})

for i in range(len(signalNew)):
    if i == len(signalNew)-1:
        break
    if signalNew.at[i, '预测'] != signalNew.at[i+1, '预测'] and signalNew.at[i, '预测'] == 0 :
        signalNew.at[i+1, '信号'] = '做多'
    if signalNew.at[i, '预测'] != signalNew.at[i+1, '预测'] and signalNew.at[i, '预测'] == 1 :
        signalNew.at[i+1, '信号'] = '做空'

if signalNew.at[0, '预测'] == 0:
    signalNew.at[0, '信号'] = '做空'
if signalNew.at[0, '预测'] == 1:
    signalNew.at[0, '信号'] = '做多'

In [ ]:
rawData['信号'] = signalNew['信号']

plt.plot(rawData['close'], label='真实值')

buy = rawData[rawData['信号'] == '做多']
sell = rawData[rawData['信号'] == '做空']

plt.scatter(buy.index.tolist(), buy['close'], label='做多', s=10, color='red')
plt.scatter(sell.index.tolist(), sell['close'], label='做空', s=10, color='green')

In [ ]:
from datetime import datetime
import backtrader as bt
from 指标计算 import *

# 创建一个策略的子类来定义指标和逻辑
class SmaCross(bt.Strategy):
    # 可配置的策略参数列表
    params = dict(
        pfast=3,  # 快速移动平均线的周期
        pslow=5   # 慢速移动平均线的周期
    )

    def __init__(self):
        # sma1 = bt.ind.SMA(period=self.p.pfast)  # 快速移动平均线
        # sma2 = bt.ind.SMA(period=self.p.pslow)  # 慢速移动平均线
        # self.crossover = bt.ind.CrossOver(sma1, sma2)  # 交叉信号指标
        self.signal = signalNew['信号']

    def next(self):
        day = len(self) - 1
        print('当前方向', self.signal[day])
        # 不在市场中
        if not self.position:
            # 通用指定
            if self.signal[day] == '做多':
                self.buy()
            if self.signal[day] == '做空':
                self.sell()
        # 在市场中
        if self.position:
            if self.position.size > 0 and self.signal[day] == '做空':
                self.close()
                self.sell()
            if self.position.size < 0 and self.signal[day] == '做多':
                self.close()
                self.buy()



cerebro = bt.Cerebro()  # 创建一个"Cerebro"引擎实例

# 创建数据源
data = cal_main('../DB/GFEX.csv', 'SI')
data['date'] = pd.to_datetime(data['date'], format='%Y%m%d')
data = data.set_index('date')
start_date = datetime(2022, 12, 22)  # 回测开始时间
end_date = datetime(2023, 4, 20)  # 回测结束时间
data = bt.feeds.PandasData(dataname=data, fromdate=start_date, todate=end_date)

cerebro.adddata(data)  # 添加数据源
cerebro.broker.setcash(1000000.0)
cerebro.broker.setcommission(commission=0.001)
cerebro.addsizer(bt.sizers.FixedSize, stake=18)
cerebro.addstrategy(SmaCross)  # 添加交易策略
cerebro.run()  # 运行回测
cerebro.plot()  # 使用一个命令进行绘图